In [1]:
import numpy as np
import pprint
import ray

ray.init()

2023-04-06 18:02:40,827	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.9.16
Ray version:,2.3.1


In [2]:
from multiagent_env import GridWorldEnv
# Import a Trainable (one of RLlib's built-in algorithms):
# We use the PPO algorithm here b/c its very flexible wrt its supported
# action spaces and model types and b/c it learns well almost any problem.
from ray.rllib.agents.ppo import PPOTrainer, PPOTF1Policy, PPOTF2Policy, PPOTorchPolicy

env = GridWorldEnv()

policies = {
    "policy_1": (PPOTF1Policy, env.observation_space, env.action_space, {}),
}

policy_mapping_fn = lambda agent_id, episode, worker, **kwargs: "policy_1"


In [3]:
config = {
    "env": GridWorldEnv,  # "my_env" <- if we previously have registered the env with `tune.register_env("[name]", lambda config: [returns env object])`.
    "env_config": {},
    "framework": "tf",  # If users have chosen to install torch instead of tf.
    "create_env_on_driver": True,
    "multiagent": {
        "policies": policies,
        "policy_mapping_fn": lambda agent_id, episode, worker, **kwargs: "policy_1",
    },
}
pprint.pprint(config)
# print()
# for agent in env.agents:
#     print(f"{agent} is now mapped to {policy_mapping_fn(agent)}")

rllib_trainer = PPOTrainer(config=config)

2023-04-06 18:02:45,259	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-04-06 18:02:45,286	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


{'create_env_on_driver': True,
 'env': <class 'multiagent_env.GridWorldEnv'>,
 'env_config': {},
 'framework': 'tf',
 'multiagent': {'policies': {'policy_1': (<class 'ray.rllib.algorithms.ppo.ppo_tf_policy.PPOTF1Policy'>,
                                          Dict('common': Dict('ore_positions': MultiDiscrete([100 100 100 100 100])), 'friendly': Dict('factory_position': MultiDiscrete([100]), 'robot_ore': MultiDiscrete([51 51 51]), 'robot_positions': MultiDiscrete([101 101 101])), 'opponent': Dict('factory_position': MultiDiscrete([100]), 'robot_ore': MultiDiscrete([51 51 51]), 'robot_positions': MultiDiscrete([101 101 101]))),
                                          MultiDiscrete([10 10 10]),
                                          {})},
                'policy_mapping_fn': <function <lambda> at 0x3674b2b80>}}
(RolloutWorker pid=11809) Metal device set to: Apple M1 Max
(RolloutWorker pid=11809) 
(RolloutWorker pid=11809) systemMemory: 32.00 GB
(RolloutWorker pid=11809) maxCache

(RolloutWorker pid=11808) 2023-04-06 18:02:48,744	WARNING env.py:296 -- Your MultiAgentEnv <GridWorldEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-04-06 18:02:49,532	WARNING env.py:296 -- Your MultiAgentEnv <GridWorldEnv instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-04-06 18:02:50,310	WARNING util.py:67 -- Install gputil for GPU system monitoring.


In [4]:
# 4) Run `train()` n times. Repeatedly call `train()` now to see rewards increase.
# Move on once you see (agent1 + agent2) episode rewards of 10.0 or more.
for _ in range(10):
    results = rllib_trainer.train()
    print(f"Iteration={rllib_trainer.iteration}: R(\"return\")={results['episode_reward_mean']}")

2023-04-06 18:03:06,744	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Iteration=1: R("return")=272.74125000000004
Iteration=2: R("return")=197.41812500000003
Iteration=3: R("return")=202.12099999999998
Iteration=4: R("return")=202.03750000000008
Iteration=5: R("return")=181.29900000000006


In [ ]:
obs = env.reset()
env.record()

simulation_time = 100
assert simulation_time > 1

for _ in range(simulation_time-1):
    
    random_sample_action = {
        'team_0': rllib_trainer.compute_action(obs["team_0"], policy_id="policy_1"),
        'team_1': rllib_trainer.compute_action(obs["team_1"], policy_id="policy_1"),
    }
    new_obs, reward, _, _, _ = env.step(random_sample_action)
    env.record()
env.render(framerate=2)

# with out:
#     env = MultiAgentArena()
#     obs = env.reset()
#     while True:
#         a1 = rllib_trainer.compute_action(obs["agent1"], policy_id="policy1")
#         a2 = rllib_trainer.compute_action(obs["agent2"], policy_id="policy2")    
#         obs, rewards, dones, _ = env.step({"agent1": a1, "agent2": a2})
#         out.clear_output(wait=True)
#         time.sleep(0.08)
#         env.render()
#         if dones["agent1"]:
#           break

IndexError: index 3 is out of bounds for axis 0 with size 1